In [65]:
import requests
from bs4 import BeautifulSoup
import pdfplumber
import re
import os
import pymongo
import time
import random
from dotenv import load_dotenv

In [66]:
load_dotenv()

MONGO_URI = os.getenv("MONGO_URI")
DATABASE_NAME = os.getenv("DATABASE_NAME")
COLLECTION_NAME = os.getenv("COLLECTION_NAME")

client = pymongo.MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

In [67]:
def extract_text_from_pdf(pdf_url):
    """PDF URL에서 텍스트 추출"""
    try:
        response = requests.get(pdf_url)
        response.raise_for_status()

        if not os.path.exists("./temp"):
            os.makedirs("./temp")

        with open("./temp/temp.pdf", "wb") as f:
            f.write(response.content)

        with pdfplumber.open("./temp/temp.pdf") as pdf:
            text = "".join([page.extract_text() for page in pdf.pages])

        return text

    except requests.exceptions.RequestException as e:
        print(f"PDF 다운로드 에러: {e}")
        return None
    except Exception as e:
        print(f"PDF 텍스트 추출 중 에러 발생: {e}")
        return None

In [68]:
def extract_report_data(report_url):
    """리포트 URL에서 PDF URL을 추출하고, PDF 내용을 추출하여 반환"""
    try:
        pdf_url = report_url
        return extract_text_from_pdf(pdf_url)

    except requests.exceptions.RequestException as e:
        print(f"Request 에러: {e}")
        return None
    except Exception as e:
        print(f"리포트 데이터 추출 중 에러 발생: {e}")
        return None

In [ ]:
def crawl_reports(base_url):
    """리포트 목록 페이지를 크롤링하여 리포트 정보 추출"""
    page_num = 1
    while True:
        url = f"{base_url}&page={page_num}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        table = soup.find("table", {"class": "type_1"})

        if not table:
            print("더 이상 리포트가 없습니다.")
            break

        rows = table.find_all("tr")[2:]  # 첫 두 행은 헤더이므로 제외

        if not rows:
            print("더 이상 리포트가 없습니다.")
            break

        for row in rows:
            cells = row.find_all("td")
            # print(cells)

            if not cells or all(cell.text.strip() == "" for cell in cells):
                continue

            company = cells[0].text.strip()
            # print("company:", company)
            title = cells[1].text.strip()
            # print("title:", title)
            report_url = cells[3].find("a")["href"]
            # print("report_url:", report_url)
            brokerage = cells[2].text.strip()
            # print("brokerage:", brokerage)
            date = cells[4].text.strip()
            # print("date:", date)

            report_content = extract_report_data(report_url)
            # print("report_content:", report_content)

            if report_content:
                report_data = {
                    "종목이름": company,
                    "종목코드": "035720",
                    "증권사": brokerage,
                    "제목": title,
                    "날짜": date,
                    "내용": report_content,
                }
                collection.insert_one(report_data)
                print(f"리포트 추출 성공: {title}/{date}")
            else:
                print(f"리포트 내용 추출 실패: {company}:{title}")

            time.sleep(random.uniform(1, 30))
        page_num += 1

In [ ]:
base_url = "https://finance.naver.com/research/company_list.naver?keyword=&brokerCode=&writeFromDate=&writeToDate=&searchType=itemCode&itemName=%C4%AB%C4%AB%BF%C0&itemCode=035720&x=58&y=13"
crawl_reports(base_url)

리포트 추출 성공: 카톡의 외연 확장 기대
리포트 추출 성공: 공개된 전략, 실행과 성과가 중요
리포트 추출 성공: AI 메이트 도입으로 코어 비즈니스 가속 시도
리포트 추출 성공: 카카오톡 2.0, SNS + AI Agent를 추구
리포트 추출 성공: ‘카카오톡+AI’의 청사진, 유저 지표 개선이..
리포트 추출 성공: 상승세의 초기 국면
리포트 추출 성공: 콘텐츠 부진의 무게가 무겁다
리포트 추출 성공: N이라면 상상 가능한 톡의 미래
리포트 추출 성공: 상상력을 열게 해준 실적 발표
리포트 추출 성공: 오픈AI 협업 발표. AI 네이티브 컴퍼니를 향..
리포트 추출 성공: 매니지먼트의 전략적 우위
리포트 추출 성공: 저성장 지속과 카카오톡 개편 기대
리포트 추출 성공: 성장성 회복이 관건
리포트 추출 성공: 톡비즈 성장 개선 정도가 중요
리포트 추출 성공: 인공지능 메신저에 대한 기대
리포트 추출 성공: 시장 부진 헤쳐 나갈 뾰족한 수 필요
리포트 추출 성공: 새로운 카카오톡에 기대를...
리포트 추출 성공: 카카오톡의 무거워진 어깨
리포트 추출 성공: 플랫폼과 콘텐츠, 각각의 숙제
리포트 추출 성공: 체류시간 확대 전략이 무조건 필요
리포트 추출 성공: 아직은 뚜렷한 모멘텀 없지만
리포트 추출 성공: 잃어버린 성장
리포트 추출 성공: 무거워진 본사의 어깨
리포트 추출 성공: 신규 원동력이 요구
리포트 추출 성공: 외형 성장 둔화로 멀티플 하향 조정
리포트 추출 성공: 뼈를 깎고, 살을 취했다
리포트 추출 성공: 성장성 둔화 심화
리포트 추출 성공: IF 구체화
리포트 추출 성공: 다이어트 중
리포트 추출 성공: 밋밋한 실적
리포트 추출 성공: 돌파구가 필요
리포트 추출 성공: 눈에 밟히는 콘텐츠 부진
리포트 추출 성공: 새로운 광고 성장 전략 필요
리포트 추출 성공: 시작되는 비핵심 사업 축소
리포트 추출 성공: 자회사 옥석 가리기
리포트 추출 성공: 확인이 필요한 시간
리포트 추출 성공: 톡비즈 성장 재가속 필요
리포트 추출 성공: 하반기 변화 예고


Data-loss while decompressing corrupted data


리포트 추출 성공: 경주하는 거북이
리포트 추출 성공: 확장이 우선
리포트 추출 성공: 1Q18은 과도기적 투자 집중 시기. 2Q18부터 ..
리포트 추출 성공: 신규사업 성장을 위한 투자의 기간
리포트 추출 성공: 여전히 풍부한 모멘텀
리포트 추출 성공: 수익 둔화 불가피, 외형 성장에 주목
리포트 추출 성공: 부진한 실적에도 풍부한 관전포인트
리포트 추출 성공: 1Q18은 투자의 시기. 2Q18부터 모멘텀 강화
리포트 추출 성공: 자회사 Monetization으로 주가 상승 기대
리포트 추출 성공: 성장을 위한 투자와 Monetization
리포트 추출 성공: 올해도 투자의 시기. 신사업/신기술 관련 중..
리포트 추출 성공: 모멘텀은 하반기 강세 전망
리포트 추출 성공: 외형확대와 질적인 성장
리포트 추출 성공: 수익성보다는 성장성에 초점
리포트 추출 성공: 유상증자 마무리될 때까지 불확실성 여전. 투..
리포트 추출 성공: 모바일 생태계 지배력 강화
리포트 추출 성공: 적정가치에 대한 고민
리포트 추출 성공: 미래 투자를 위한 유상증자 결정
리포트 추출 성공: GDR 발행 결정, 관건은 M&A 대상
리포트 추출 성공: 4Q17 OP QoQ (+) 전망. 마케팅비 생각보다 효..
리포트 추출 성공: 중장기 승부주. 그런데, 단기 그림도 긍정적
리포트 추출 성공: 전방위 성장모드, 이익률 개선이 관건
리포트 추출 성공: 잰 걸음
리포트 추출 성공: 아직은 투자의견 유지
리포트 추출 성공: 고른 성장의 힘! 향후 페이, 게임즈, 로엔 가..
리포트 추출 성공: 본업과 자회사 모두 긍정적
리포트 추출 성공: 사업부문별 성장과 시너지 효과
리포트 추출 성공: 장기 승부주. 단기적으론 2H17 마케팅비 급증..
리포트 추출 성공: 플랫폼의 영향력은 점점 더 커진다
리포트 추출 성공: 시작된 광고의 성장과 플랫폼의 힘 확읶
리포트 추출 성공: 시장경쟁력 회복에 대한 기대
리포트 추출 성공: 탄력이 붙은 광고
리포트 추출 성공: 2Q17 실적 호조. 종전 전망치 부합